# Experimenting with Random Model
---------------

In [1]:
import __init__
#import models.random as random_model
#import models.cross_validator as cross_validator
import models.datahandler as datahandler
from models.datamodel import DataModel
import models.outputmaker as outputmaker
import evaluator.metrics as metrics

trial = datahandler.load_train('../data/dataset/tsd_trial.csv', verbose=True)
train = datahandler.load_train('../data/dataset/tsd_train.csv', verbose=True)

/home/hb/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


In [2]:
datamodel = DataModel(model='crf')

In [3]:
from tqdm import tqdm
import pandas as pd

In [12]:
X, Y, T, texts = [], [], [], []
for i in tqdm(range(train.shape[0])):   
    x, y, taboo_words, text = datamodel.transform(train.iloc[i])    
    X.append(x)
    Y.append(y)
    T.append(taboo_words)
    texts.append(text)

100%|██████████| 7939/7939 [01:59<00:00, 66.54it/s]


In [11]:
#trl = [X[i][j] for i in range(len(X)) for j in range(len(X[i])) if X[i][j][2] == 'toxic']

In [13]:
T

['violent and aggressive immigrant',
 'fucking',
 'Damn',
 'knucklehead',
 'killing',
 '',
 'stupid emoticons',
 'Dumb',
 'idiots stupid',
 'pathetic',
 'imbecile',
 'idiot',
 'repugnant',
 'joke',
 'protest',
 'f*** you',
 'stupid',
 'sociopathic parasitic',
 'sycophants bum',
 'dumb',
 'Kill LOL the left is a joke',
 '',
 'fool',
 'ignorant',
 '',
 'Damn',
 'stupid',
 '',
 'mistake',
 'scam',
 'ignorant lunatics',
 'violent and extremist doctrines as Islamists are',
 'ignorant',
 'testicles stupid',
 "I'll shoot you if I catch you I'll put you in GRAVE YARD",
 'moron',
 'Bury',
 'Morons',
 'scum',
 'incompetent bunglers',
 'idiots',
 'background',
 'Hang the witch You are a racist',
 'stupid',
 'idiots',
 'morons',
 'stupid',
 'hypocrite ignorant hypocrisy',
 'Darn',
 'coward coward loser died',
 'shit',
 'idiotic',
 'idiots',
 'crazed',
 'crap',
 'stupid stupid',
 'demeans nuts',
 'challenging',
 'These Immoral',
 'idiot',
 '',
 'three black men in America is a felon',
 'stupid jerk

In [23]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import spacy
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import MaxAbsScaler

__vectors__ = {}

class MLModels:
    
    def __init__(self, model_name='pasagclf'):
        '''
        Initialize MLModels model
        '''
        self.model_name = model_name
        self.nlp = spacy.load('en_core_web_lg')
        if self.model_name == 'pasagclf':
            self.model = LogisticRegression()
        self.func_x = lambda x:[x[i][0] for i in range(len(x))]
        self.func_y = lambda x:[x[i][2] for i in range(len(x))]
        self.F_simple = lambda word: self.nlp(word).vector
        self.max_abs_scaler = MaxAbsScaler()
        
    def get_features(self, X):
        features = []
        for i in tqdm(range(len(X))):
            for x in self.func_x(X[i]):
                try:
                    features.append(__vectors__[x])
                except:
                    feature = self.F_simple(x)
                    __vectors__[x] = feature
                    features.append(__vectors__[x])
        return features
    
    def fit(self, X, Y=None):
        '''
            Train ML model
        Arguments:
            X: a train set with input format of CRF
            Y: None, just to follow the fit architecture
        Returns: None
        '''
        self.__init__(model_name=self.model_name)
        X_train = self.get_features(X)
        Y_train = [x for i in range(len(X)) for x in self.func_y(X[i])]
        self.max_abs_scaler.fit(X_train)
        self.model.fit(self.max_abs_scaler.transform(X_train), Y_train)
        return X_train, Y_train
        
    def predict(self, X):
        '''
            Make Prediction using pretrained ML model
        Arguments:
            X: a test set with input format of CRF
        Returns: 
            predicts
        '''
        preds = []
        for X_ in tqdm(X):
            X_test_data = [__vectors__[x] if x in __vectors__ else list(self.F_simple(x)) for x in self.func_x(X_)]
            preds.append(self.model.predict(self.max_abs_scaler.transform(X_test_data)))
        return preds
    
    def predict_proba(self, X):
        '''
            Make prediction probability using pretrained CRF model
        Arguments:
            X: a test set with input format of CRF
        Returns: 
            predict_probas
        '''
        preds = []
        for X_ in X:
            X_test_data = [__vectors__[x] if x in __vectors__ else list(self.F_simple(x)) for x in self.func_x(X_)]
            preds.append(self.model.predict_proba(self.max_abs_scaler.transform(X_test_data)))
        return preds

In [24]:
model = MLModels()

In [25]:
xx, yy = model.fit(X[:50])

100%|██████████| 50/50 [00:08<00:00,  5.58it/s]


In [26]:
preds = model.predict(X[:50])

100%|██████████| 50/50 [00:00<00:00, 373.87it/s]


In [36]:
outs = outputmaker.crf_output(X[:50], texts[:50], preds)
index = 3
print(X[index])
print(texts[index])
print(outs[index])

[('Trump', 'PROPN', 'normal'), ('Claimed', 'VERB', 'normal'), ('that', 'SCONJ', 'normal'), ('Russia', 'PROPN', 'normal'), ('will', 'VERB', 'normal'), ('never', 'ADV', 'normal'), ('invade', 'VERB', 'normal'), ('the', 'DET', 'normal'), ('Ukraine', 'PROPN', 'normal'), (',', 'PUNCT', 'normal'), ('when', 'ADV', 'normal'), ('Russia', 'PROPN', 'normal'), ('already', 'ADV', 'normal'), ('has', 'AUX', 'normal'), ('-', 'PUNCT', 'normal'), ('how', 'ADV', 'normal'), ('stupid', 'ADJ', 'toxic'), ('can', 'VERB', 'normal'), ('people', 'NOUN', 'normal'), ('be', 'AUX', 'normal'), ('?', 'PUNCT', 'normal')]
Trump Claimed that Russia will never invade the Ukraine, when Russia already has - how stupid can people be?
[81, 87, 88, 89, 90, 91, 92]


In [37]:
len(texts[index])

108

In [38]:
s = texts[index]

In [41]:
s[87:92+1]

'stupid'